In [2]:
## RECOMMENDATION ENGINE using customer-item matrix(collaborative filtering)

In [3]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [4]:
# IMPORTING THE DATA USIN DATAFRAME

In [5]:
df = pd.read_csv("C:\\Users\\DELL\\Documents\\MIR\\Recommender1.csv",encoding='ISO-8859-1')

In [6]:
df.head(5)

,Age,MaritalStatus,Gender,YearlyIncome,TotalChildren,NumberChildrenAtHome,NumberCarsOwned,EnglishEducation,EnglishOccupation,HouseOwnerFlag,CommuteDistance,ProductKey,CustomerKey,EnglishProductCategoryName,EnglishProductSubcategoryName,EnglishProductName,OrderQuantity,Region,OrderDate,DateFirstPurchase
0,43,M,M,90000,2,0,0,Bachelors,Professional,1,1-2 Miles,353,11000,Bikes,Mountain Bikes,"Mountain-200 Silver, 38",1,Australia,1/18/2013 0:00,1/19/2011 0:00
1,43,M,M,90000,2,0,0,Bachelors,Professional,1,1-2 Miles,485,11000,Accessories,Fenders,Fender Set - Mountain,1,Australia,1/18/2013 0:00,1/19/2011 0:00
2,43,M,M,90000,2,0,0,Bachelors,Professional,1,1-2 Miles,530,11000,Accessories,Tires and Tubes,Touring Tire Tube,1,Australia,5/3/2013 0:00,1/19/2011 0:00
3,43,M,M,90000,2,0,0,Bachelors,Professional,1,1-2 Miles,344,11000,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",1,Australia,1/19/2011 0:00,1/19/2011 0:00
4,43,M,M,90000,2,0,0,Bachelors,Professional,1,1-2 Miles,488,11000,Clothing,Jerseys,"Short-Sleeve Classic Jersey, S",1,Australia,5/3/2013 0:00,1/19/2011 0:00


In [7]:
df.shape # CHECKING THE SHAPE OF THE DATAFRAME

(60398, 20)

In [8]:
df.dtypes # CHECKING DATA TYPES

Age                               int64
MaritalStatus                    object
Gender                           object
YearlyIncome                      int64
TotalChildren                     int64
NumberChildrenAtHome              int64
NumberCarsOwned                   int64
EnglishEducation                 object
EnglishOccupation                object
HouseOwnerFlag                    int64
CommuteDistance                  object
ProductKey                        int64
CustomerKey                       int64
EnglishProductCategoryName       object
EnglishProductSubcategoryName    object
EnglishProductName               object
OrderQuantity                     int64
Region                           object
OrderDate                        object
DateFirstPurchase                object
dtype: object

In [9]:
df.columns

Index(['Age', 'MaritalStatus', 'Gender', 'YearlyIncome', 'TotalChildren',
       'NumberChildrenAtHome', 'NumberCarsOwned', 'EnglishEducation',
       'EnglishOccupation', 'HouseOwnerFlag', 'CommuteDistance', 'ProductKey',
       'CustomerKey', 'EnglishProductCategoryName',
       'EnglishProductSubcategoryName', 'EnglishProductName', 'OrderQuantity',
       'Region', 'OrderDate', 'DateFirstPurchase'],
      dtype='object')

In [10]:
df.isnull().sum() #count of null values in columns

Age                              0
MaritalStatus                    0
Gender                           0
YearlyIncome                     0
TotalChildren                    0
NumberChildrenAtHome             0
NumberCarsOwned                  0
EnglishEducation                 0
EnglishOccupation                0
HouseOwnerFlag                   0
CommuteDistance                  0
ProductKey                       0
CustomerKey                      0
EnglishProductCategoryName       0
EnglishProductSubcategoryName    0
EnglishProductName               0
OrderQuantity                    0
Region                           0
OrderDate                        0
DateFirstPurchase                0
dtype: int64

In [11]:
columns_to_copy = ['ProductKey','CustomerKey','OrderQuantity','EnglishProductName']

In [12]:
RH = df[columns_to_copy].copy()

In [13]:
RH.head(5)

,ProductKey,CustomerKey,OrderQuantity,EnglishProductName
0,353,11000,1,"Mountain-200 Silver, 38"
1,485,11000,1,Fender Set - Mountain
2,530,11000,1,Touring Tire Tube
3,344,11000,1,"Mountain-100 Silver, 38"
4,488,11000,1,"Short-Sleeve Classic Jersey, S"


In [14]:
RH.shape

(60398, 4)

In [15]:
# Create a customer-product matrix using pivot_table
customer_product_matrix = RH.pivot_table(index='CustomerKey', columns='EnglishProductName', values='OrderQuantity', fill_value=0)


In [16]:
# Display the customer-product matrix
print("Customer-Product Matrix:")
print(customer_product_matrix)

Customer-Product Matrix:
EnglishProductName  AWC Logo Cap  All-Purpose Bike Stand  \
CustomerKey                                                
11000                          0                       0   
11001                          1                       0   
11002                          0                       0   
11003                          1                       0   
11004                          0                       0   
...                          ...                     ...   
29479                          0                       0   
29480                          1                       0   
29481                          0                       0   
29482                          0                       0   
29483                          0                       0   

EnglishProductName  Bike Wash - Dissolver  Classic Vest, L  Classic Vest, M  \
CustomerKey                                                                   
11000                               

In [17]:
# Transpose the user-item matrix for item-based collaborative filtering
product_customer_matrix = customer_product_matrix.T

In [18]:
product_customer_matrix

CustomerKey,11000,11001,11002,11003,11004,11005,11006,11007,11008,11009,...,29474,29475,29476,29477,29478,29479,29480,29481,29482,29483
EnglishProductName,,,,,,,,,,,,,,,,,,,,,
AWC Logo Cap,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
All-Purpose Bike Stand,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bike Wash - Dissolver,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Classic Vest, L",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Classic Vest, M",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Touring-3000 Yellow, 62",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Water Bottle - 30 oz.,0,1,0,1,0,0,1,0,1,0,...,0,0,0,0,1,0,1,0,0,0
"Women's Mountain Shorts, L",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
# Compute item similarity matrix using cosine similarity
product_similarity_matrix = cosine_similarity(product_customer_matrix)

In [21]:
product_similarity_matrix

array([[1.        , 0.03751172, 0.04319708, ..., 0.1023047 , 0.10158223,
        0.0931603 ],
       [0.03751172, 1.        , 0.02819267, ..., 0.00673401, 0.00683842,
        0.01839628],
       [0.04319708, 0.02819267, 1.        , ..., 0.02661546, 0.0216225 ,
        0.01551133],
       ...,
       [0.1023047 , 0.00673401, 0.02661546, ..., 1.        , 0.        ,
        0.        ],
       [0.10158223, 0.00683842, 0.0216225 , ..., 0.        , 1.        ,
        0.        ],
       [0.0931603 , 0.01839628, 0.01551133, ..., 0.        , 0.        ,
        1.        ]])

In [22]:
# Function to recommend products for a customer using item-based collaborative filtering
def recommend_products_item_based(customer_product_matrix, product_similarity_matrix, CustomerKey):
    # Predicted products: weighted sum of products based on similarity
    predicted_products = product_similarity_matrix.dot(customer_product_matrix.loc[CustomerKey])
    
    # Convert the predicted products to a pandas Series
    predicted_products_series = pd.Series(predicted_products, index=customer_product_matrix.columns)
    
    # Exclude products the customer has already interacted with
    interacted_products = customer_product_matrix.columns[customer_product_matrix.loc[CustomerKey] == 1]
    recommended_products = predicted_products_series.drop(interacted_products, errors='ignore')
    
   # Sort the remaining products in descending order of predicted scores
    recommended_products = recommended_products.sort_values(ascending=False)
    
     # Get the top N recommended products
    top_n_products = recommended_products.head(5)
    
    return top_n_products.index.tolist()






In [23]:
# Recommend products for a specific customer (customer 29480) using item-based collaborative filtering
recommended_products = recommend_products_item_based(customer_product_matrix, product_similarity_matrix, 29480)
print("Recommended Products for Customer 29480:")
print(recommended_products)

Recommended Products for Customer 29480:
['Mountain Bottle Cage', 'HL Mountain Tire', 'Sport-100 Helmet, Red', 'Mountain Tire Tube', 'Fender Set - Mountain']


In [24]:
customer_product_matrix.loc[29480] # checking for the products purchased by Customer 29480

EnglishProductName
AWC Logo Cap                  1
All-Purpose Bike Stand        0
Bike Wash - Dissolver         0
Classic Vest, L               0
Classic Vest, M               0
                             ..
Touring-3000 Yellow, 62       0
Water Bottle - 30 oz.         1
Women's Mountain Shorts, L    0
Women's Mountain Shorts, M    0
Women's Mountain Shorts, S    0
Name: 29480, Length: 130, dtype: int64

In [25]:
purchased_product = customer_product_matrix.loc[29480][customer_product_matrix.loc[29480]> 0]
purchased_product

EnglishProductName
AWC Logo Cap               1
Road Bottle Cage           1
Sport-100 Helmet, Black    1
Touring-1000 Yellow, 50    1
Water Bottle - 30 oz.      1
Name: 29480, dtype: int64

In [26]:
purchased_products_list = purchased_product.index.tolist() # converting to a list

In [27]:
purchased_products_list # Output the List

['AWC Logo Cap',
 'Road Bottle Cage',
 'Sport-100 Helmet, Black',
 'Touring-1000 Yellow, 50',
 'Water Bottle - 30 oz.']

In [28]:
# checking the accuracy of the Recommendation Engine

In [29]:
# defining the MRR function
def calculate_mrr(purchased_product, recommended_products):
    """
    Calculate Mean Reciprocal Rank (MRR) for a set of recommendations.

    Parameters:
    - actual: List of lists, where each inner list contains the actual items for a user.
    - predicted: List of lists, where each inner list contains the predicted items for a user.

    Returns:
    - MRR score.
    """
    mrr_sum = 0.0

    for purchased_product, recommended_products in zip(purchased_product, recommended_products):
        # Check if there is any overlap between actual and predicted items
        common_items = set(purchased_product).intersection(recommended_products)

        # If there are common items, calculate the reciprocal rank
        if common_items:
            rank = min([recommended_products.index(item) + 1 for item in common_items])
            mrr_sum += 1.0 / rank

    # Calculate the mean reciprocal rank
    mrr = mrr_sum / len(purchased_products_list) if len(purchased_products_list) > 0 else 0.0

    return mrr


In [30]:
# Calculating the MRR for the customer with customerKey 29480:
mrr_score = calculate_mrr(purchased_products_list, recommended_products)
print(f"Mean Reciprocal Rank (MRR): {mrr_score}")

Mean Reciprocal Rank (MRR): 0.5666666666666667


In [31]:
#A Mean Reciprocal Rank (MRR) of approximately 0.567 is quite good and suggests that, on average, the first correct recommendation is found within the top two positions of the recommended list. This indicates a strong performance of the recommendation system in terms of ranking relevant items for users.

In [32]:
# Using SVD to train and test the model

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [35]:
train_data, test_data = train_test_split(customer_product_matrix, test_size=0.5, random_state=42)

In [36]:
k = 10  # Number of latent factors
svd = TruncatedSVD(n_components=k)
user_factors = svd.fit_transform(train_data)
item_factors = svd.components_


In [37]:
# Reconstruct the customer_product_matrix using the factors
predicted_ratings= np.dot(user_factors, item_factors)

In [38]:
predicted_ratings

array([[ 0.05113106,  0.00091393,  0.00045752, ...,  0.00241232,
         0.00213235,  0.00256865],
       [-0.01108115,  0.00175671, -0.00438826, ...,  0.0014292 ,
         0.00410291,  0.00153213],
       [-0.00554322, -0.00584896,  0.00715873, ...,  0.01123982,
         0.01597542,  0.00482711],
       ...,
       [-0.01692615,  0.01029606,  0.00716285, ..., -0.00114239,
        -0.00131707, -0.00103118],
       [ 0.01508723,  0.0069978 ,  0.01250521, ...,  0.00691296,
         0.00617477,  0.0046099 ],
       [ 0.02257353,  0.00248009, -0.00467271, ...,  0.00306381,
         0.00535378,  0.00317721]])

In [39]:
train_data.index

Int64Index([27712, 21912, 26891, 20605, 21620, 26192, 28353, 14003, 19215,
            26364,
            ...
            22363, 25423, 15426, 27850, 17265, 22284, 22964, 16390, 11860,
            26795],
           dtype='int64', name='CustomerKey', length=9242)

In [40]:
train_data.columns

Index(['AWC Logo Cap', 'All-Purpose Bike Stand', 'Bike Wash - Dissolver',
       'Classic Vest, L', 'Classic Vest, M', 'Classic Vest, S',
       'Fender Set - Mountain', 'HL Mountain Tire', 'HL Road Tire',
       'Half-Finger Gloves, L',
       ...
       'Touring-3000 Blue, 62', 'Touring-3000 Yellow, 44',
       'Touring-3000 Yellow, 50', 'Touring-3000 Yellow, 54',
       'Touring-3000 Yellow, 58', 'Touring-3000 Yellow, 62',
       'Water Bottle - 30 oz.', 'Women's Mountain Shorts, L',
       'Women's Mountain Shorts, M', 'Women's Mountain Shorts, S'],
      dtype='object', name='EnglishProductName', length=130)

In [41]:
# Convert the reconstructed matrix back to a DataFrame
predicted_customer_product_df = pd.DataFrame(predicted_ratings, index=train_data.index, columns=train_data.columns)

In [42]:
predicted_customer_product_df

EnglishProductName,AWC Logo Cap,All-Purpose Bike Stand,Bike Wash - Dissolver,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Fender Set - Mountain,HL Mountain Tire,HL Road Tire,"Half-Finger Gloves, L",...,"Touring-3000 Blue, 62","Touring-3000 Yellow, 44","Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S"
CustomerKey,,,,,,,,,,,,,,,,,,,,,
27712,0.051131,0.000914,0.000458,0.000528,0.000745,0.000406,0.018755,0.003587,0.001794,0.001727,...,0.000417,0.000592,0.000437,0.000525,0.000232,0.000306,0.014598,0.002412,0.002132,0.002569
21912,-0.011081,0.001757,-0.004388,0.004775,0.004880,0.008223,0.050194,0.040709,0.026798,0.036246,...,0.000596,0.003221,0.001789,0.001130,0.003810,0.004247,0.041896,0.001429,0.004103,0.001532
26891,-0.005543,-0.005849,0.007159,0.002113,0.000782,0.007779,-0.004072,0.037259,0.004446,0.067854,...,0.004531,0.008041,0.003760,0.003663,0.005484,0.007452,-0.025357,0.011240,0.015975,0.004827
20605,0.034493,0.043154,0.018297,0.015024,0.021756,0.013070,0.020258,0.059332,0.327305,0.057387,...,0.001710,0.002249,0.000554,-0.000200,-0.002276,-0.001556,-0.012009,0.007175,0.002663,0.007135
21620,0.009732,0.000462,0.026583,0.007466,0.004685,0.004031,0.062671,-0.014375,-0.048857,0.013377,...,0.008705,0.004402,0.009500,0.004571,0.003459,0.003028,0.355685,0.003033,0.004049,0.002680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22284,0.043143,0.042987,0.019108,0.014570,0.020373,0.012279,0.000547,0.047448,0.319744,0.052963,...,0.002138,0.002330,0.001201,-0.000042,-0.002213,-0.001648,0.002009,0.007194,0.002805,0.007496
22964,0.082205,0.001026,0.002287,0.000075,0.000155,0.000008,0.016605,0.010249,-0.002326,0.000825,...,0.000422,0.000801,0.000378,0.000672,0.000214,0.000342,0.009656,0.004221,0.003597,0.004165
16390,-0.016926,0.010296,0.007163,0.002729,0.004524,0.003137,-0.004990,-0.000008,0.073851,0.007606,...,-0.000827,-0.000594,-0.000761,-0.000589,-0.000363,-0.000293,0.013197,-0.001142,-0.001317,-0.001031


In [43]:
# Evaluate performance
mse = mean_squared_error(test_data.values.flatten(), predicted_ratings.flatten())
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 0.18084930721174675


In [44]:
#The Root Mean Squared Error (RMSE). An RMSE of 0.1808 suggests that the model's predictions are, on average, close to the actual values.